In [1]:
import pandas as pd
import datetime as datetime
import numpy as np
import copy
from RSU_functions import * 

In [2]:
# If you want to change your marginal income tax (above 72k of taxable income, it is at 41%). Not really needed
tax_info_dict = {'TMI_IR': 0.3, 'cotisation': 0.172, 'flat_tax_plus_value': 0.3, 'deductible_csg':0.0680}

# Put here all sales and vesting event from EquatePlus
please put all event until end of 2022, including sales / vesting from before (other years). If you have sales in 2020 or 2021 either rely on the built in function or please du check yourself how you reported them before. 

If you sold everything you had at one point, before 2021 then you can put only vesting and transaction after this big sale event.

If you sold everything in 2021, please put all information in 2021 and even before if you want the CSG rebate computation, if you don't, you may start with all events starting after this "big sales". Please note this CSG rebate should be prefilled, so it is only to check.

If you did not sell everything, please find out which stock you declared selling in your previous declaration (for stock sold before this year of 2022) and add the amount sold to the vesting event as amount_already_declared

For sales, you need to add the stock unit price. In equate plus, go to the bibliotheque menu, then transaction. For sales, click on the date which is clickable and then report "prix de vente" IN USD (VERY IMPORTANT).

For Macron law, only stock with grant date later than the 25th of june 2020 are Macron 3. Indeed (ridiculously) the date of interest for the macron law 3 to kick in is not the grant date, but the date of the AG autorising the grant. Sadly all of our grants coming before the June 25th 2020 are NOT macron law eligible. Should you decide to have your (wrong) interpretation of the law, it should work though.

If you sold more than 300k this year, please note that the macron law 1 or 2 differs. This notebook is not handling it correctly.

In [15]:

vesting_event = [#each row for a vesting. If you have more vesting, please copy the full line with the comma at the end, and then put your value on each row. 
    # You should fill just the date and the amount. Please let macron_law_id at 0, we do not have newer RSU. If you decide 
    # to have your personnal interpretation of the law, or have non-criteo RSU, please put 3 instead.
                 {'date':datetime.datetime(2019,12,13),'amount':225,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':225},
                {'date':datetime.datetime(2020,3,13),'amount':28,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':28},
                {'date':datetime.datetime(2020,6,15),'amount':29,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':29},
                {'date':datetime.datetime(2020,9,14),'amount':28,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':28},
                {'date':datetime.datetime(2020,12,14),'amount':28,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':28},
    # amount sold is only for vested stock that you sold and declared in the years before. Here the previous stock was sold the 16 of december
    # so it is the last that I am filling. It is not necessary to fill it if you sold everything you had at your last sale event last year
    # here it should therefore do nothing because in this example, all stocks were sold. In case of doubt, just fill it.
    # When filled, these stocks are sold in priority over all else at the next transaction.
    # If I find stocks that were supposedly declared in 2021 that are sold in 2022, I will raise an error and ask you to check what you wrote. 
                {'date':datetime.datetime(2021,3,15),'amount':28,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2021,6,14),'amount':28,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2021,7,26),'amount':600,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2021,9,13),'amount':28,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2021,10,25),'amount':75,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2021,12,13),'amount':28,'macron_law_id':0, 'event':'vesting'},
                    {'date':datetime.datetime(2022,1,25),'amount':75,'macron_law_id':0, 'event':'vesting'},
                    {'date':datetime.datetime(2022,4,25),'amount':75,'macron_law_id':0, 'event':'vesting'},
                    {'date':datetime.datetime(2022,6,22),'amount':250,'macron_law_id':0, 'event':'vesting'},
                    {'date':datetime.datetime(2022,7,25),'amount':75,'macron_law_id':0, 'event':'vesting'},
                    {'date':datetime.datetime(2022,9,22),'amount':312,'macron_law_id':0, 'event':'vesting'},
    #     I PUT macron law id 3 for the last vesting taht is sold (the next one in december is not sold) to show you how to put it

                    {'date':datetime.datetime(2022,10,25),'amount':75,'macron_law_id':3, 'event':'vesting'},
                    {'date':datetime.datetime(2022,12,22),'amount':313,'macron_law_id':0, 'event':'vesting'}


                ]

selling_event = [
                 #each row for a sale. If you have more sale, please copy the full line with the comma at the end, and then put your value on each row. 
    # You should just fill the date, amount, and stock_unit_price_USD, and total_fee_dollars 
    {'date':datetime.datetime(2020,8,3),'amount':282, 'event':'sale', 'stock_unit_price_USD':13.62, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2020,12,16),'amount':56, 'event':'sale', 'stock_unit_price_USD':18.80, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2021,3,16),'amount':28, 'event':'sale', 'stock_unit_price_USD':33.37, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2021,8,6),'amount':628, 'event':'sale', 'stock_unit_price_USD':39.18, 'total_fee_dollars':108.26},
    {'date':datetime.datetime(2021,9,14),'amount':28, 'event':'sale', 'stock_unit_price_USD':36.72, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2021,11,5),'amount':75, 'event':'sale', 'stock_unit_price_USD':40.57, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2021,12,16),'amount':28, 'event':'sale', 'stock_unit_price_USD':40.71, 'total_fee_dollars':38.5},
        {'date':datetime.datetime(2022,5,6),'amount':50, 'event':'sale', 'stock_unit_price_USD':26.00, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2022,8,8),'amount':30, 'event':'sale', 'stock_unit_price_USD':26.821, 'total_fee_dollars':65.71},
    {'date':datetime.datetime(2022,8,10),'amount':75, 'event':'sale', 'stock_unit_price_USD':27.478, 'total_fee_dollars':93.7},
        {'date':datetime.datetime(2022,8,15),'amount':320, 'event':'sale', 'stock_unit_price_USD':27.478, 'total_fee_dollars':93.7},
    {'date':datetime.datetime(2022,11,3),'amount':387, 'event':'sale', 'stock_unit_price_USD':24.845, 'total_fee_dollars':47.11},

]

In [16]:
total_amount = 0
for ev in selling_event:
    if ev['date'].year == 2022:
        total_amount +=ev['amount'] * ev['stock_unit_price_USD']
print('Your total sales amount in dollars in 2022 is equal to {}. This is for sanity check'.format(total_amount))

Your total sales amount in dollars in 2022 is equal to 22573.455. This is for sanity check


Computation occurs here, please just run these cells

In [17]:
all_events_sorted = sorted(vesting_event + selling_event, key = lambda k: k['date'])

In [18]:
portfolio = {}
portfolio['available_stock'] = []
portfolio['sale_event_recap'] = []
for event in all_events_sorted:
    if event['event'] == 'vesting':
        portfolio['available_stock'].append(copy.deepcopy(event))
    if event['event'] == 'sale':
        available_stock_after_transaction, sale_event_recap = get_sales_result(event, portfolio,
                                                                               matching_method = get_sale_order_from_optionality)
        portfolio['available_stock'] = available_stock_after_transaction
        portfolio['sale_event_recap'].append(sale_event_recap)
    if event['date'].year == 2022:
        sum_stock_supposed_to_be_sold = 0
        for stock in portfolio['available_stock']:
            sum_stock_supposed_to_be_sold += stock.get('amount_already_declared', 0)
        if sum_stock_supposed_to_be_sold> 0:
            raise Exception(
            '''
            you have stock that you already declared in the previous years that are getting sold in 2022. Please recheck
            your inputs. If you declared them, they should have been sold in the past.            
            ''')
        

In [19]:
available_stock_after_transaction

[{'date': datetime.datetime(2022, 12, 22, 0, 0),
  'amount': 313,
  'macron_law_id': 0,
  'event': 'vesting'}]

# Results to put in your taxation returns
The following cells prints all transaction for the form No 2074. 
For the cell 511 mettre Action criteo et computershare.
Vesting amount with moins value, rebate etc... are for debugging purposes, please look if they make sense.

To explain quickly what we do here, for each sale, you are taxed on your vesting amount (the value of the stock the day you vest them) as if it was income and with the 30% flat tax for any capital gain on top of the vesting amount.

In order to avoid to tax you on money you do not have, if you have a capital loss, instead of declaring it as a capital loss, it is removed from your vesting amount and in this case, your selling amount(money you get from your sales) becomes your vesting amount.

Then, because this is france, we created tax rebate that depends on if the stock was granted to you after the 25th of june 2020 (in which case you only pay income tax on 50% of the vesting gain for reasons) of, if it is granted before this date, it depends if you have kept the stock two years after the vesting, and then you get the 50% rebate as for macron 3.

Because it was too easy, there is a specificity if you sell more than 300k. because I am not paid for this, it is NOT well handled in this notebook and it won't until I am myself in this situation (soon I hope).

This explains all the complex computation below.

In [20]:
tax = 0
Capital_gain = 0
vesting_gain_before_rebate = 0
rebate_amount_macron_3 = 0
rebate_amount_from_detention = 0
total_selling_price = 0
csg_rebate = 0
Capital_gain_2021 = 0
rebate_amount_2021 = 0
vesting_gain_before_rebate_2021 = 0
for event in portfolio['sale_event_recap']:
    for ev in event:            
        if ev['date de la cession (513)'].year ==2022:
            tax +=ev['tax']
            total_selling_price += ev['prix de cession net (518)']
            vesting_gain_before_rebate += ev['vesting_amount_with_moins_value']

            if ev['macron_law_id'] == 3:
                rebate_amount_macron_3 += ev['rebate_with_moins_value']
            else:
                rebate_amount_from_detention += ev['rebate_with_moins_value']

            if ev['resultat'] > 0:
                print('\n ----------------NOUVELLE TRANSACTION ---------- \n')
                for key, value in ev.items():
                    print( key, value)
                Capital_gain += ev['resultat']
            
        if ev['date de la cession (513)'].year ==2021:
            #here it is to compute CSG than can be dedudected from last year, box 6DE. Only what is 
            # subject to IR can be deducted, hence only the  vesting amount after rebate and moins value
            #will be deductible
            if ev['resultat'] > 0:
                Capital_gain_2021 += ev['resultat']

            rebate_amount_2021 += ev['rebate_with_moins_value']
            vesting_gain_before_rebate_2021 += ev['vesting_amount_with_moins_value']
        



 ----------------NOUVELLE TRANSACTION ---------- 

date de la cession (513) 2022-08-08 00:00:00
valeur unitaire de la cession (514) 26.2976762427689
nombre de titres cedes (515) 30
montant global (516) 788.9302872830671
frais de cession (517) 64.4278850867732
prix de cession net (518) 724.5024021962938
prix ou valeur acquisition unitaire (520) 22.89099526066351
prix daquisition global (521 et 523) 686.7298578199053
resultat 37.77254437638851
vesting_amount_with_moins_value 686.7298578199053
rebate_with_moins_value 0.0
rebate 0
tax 354.7966217299438
macron_law_id 0

 ----------------NOUVELLE TRANSACTION ---------- 

date de la cession (513) 2022-08-10 00:00:00
valeur unitaire de la cession (514) 26.802575107296143
nombre de titres cedes (515) 75
montant global (516) 2010.1931330472107
frais de cession (517) 91.39680062426845
prix de cession net (518) 1918.7963324229422
prix ou valeur acquisition unitaire (520) 22.89099526066351
prix daquisition global (521 et 523) 1716.8246445497632
re

# For debugging / your interest only
The following cell shows all transaction with a loss. You don't have to report them in the plus or moins value, but they have been used to compute values in form 2042 C and are shown here if you want to check that there isn\t any mistake or understand better the process

In [21]:
for event in portfolio['sale_event_recap']:
    for ev in event:            
        if ev['date de la cession (513)'].year ==2022:
            if ev['resultat'] < 0:
                print('\n ----------------NOUVELLE TRANSACTION ---------- \n')
                for key, value in ev.items():
                    print(  key, value)


 ----------------NOUVELLE TRANSACTION ---------- 

date de la cession (513) 2022-05-06 00:00:00
valeur unitaire de la cession (514) 24.597918637653738
nombre de titres cedes (515) 50
montant global (516) 1229.895931882687
frais de cession (517) 36.42384105960265
prix de cession net (518) 1193.4720908230843
prix ou valeur acquisition unitaire (520) 29.605963791267303
prix daquisition global (521 et 523) 1480.2981895633652
resultat -286.82609874028094
vesting_amount_with_moins_value 1229.895931882687
rebate_with_moins_value 0.0
rebate 0
tax 580.5108798486282
macron_law_id 0

 ----------------NOUVELLE TRANSACTION ---------- 

date de la cession (513) 2022-08-15 00:00:00
valeur unitaire de la cession (514) 26.95242766061795
nombre de titres cedes (515) 25
montant global (516) 673.8106915154488
frais de cession (517) 22.976949485041686
prix de cession net (518) 650.833742030407
prix ou valeur acquisition unitaire (520) 29.605963791267303
prix daquisition global (521 et 523) 740.14909478168

Normally here you should have automatically computed the sum of "plus value", and it should be equal to the value printed in the cell below. 
For all transaction resulting in a moins value, I removed the moins value from the vesting price to redue the taxes

In [22]:
print('The total capital gain is {}'.format(Capital_gain))

The total capital gain is 1335.8859339530106


In [23]:
vesting_gain_after_rebate_and_moins_value = vesting_gain_before_rebate - rebate_amount_macron_3 - rebate_amount_from_detention
total_tax_to_pay_approx = vesting_gain_after_rebate_and_moins_value * (tax_info_dict['TMI_IR']) + vesting_gain_before_rebate * tax_info_dict[
        'cotisation'] + Capital_gain * tax_info_dict[
                     'flat_tax_plus_value']


In [24]:
if vesting_gain_after_rebate_and_moins_value > 300000:
    print('fill 1TT, form 2042 C with ', vesting_gain_after_rebate_and_moins_value - 300000 )
print('fill 1TZ, form 2042 C with ', vesting_gain_after_rebate_and_moins_value)
print('fill 1UZ abbatement pour duree de detention, form 2042 C with the rebate in euro', rebate_amount_from_detention )
print('fill 1WZ abattement de 50%, form 2042 C with the rebate in euro', rebate_amount_macron_3 )

print('fill 3VG , form 2042 C with capital gain', Capital_gain)
print('fill 3VH , form 2042 C with capital loss', 0, 'except if you have other sources of capital losses. You fill 0 because you are removing these losses from the 1TZ field' )


print('You sold for {} euros in stocks'.format(np.round(total_selling_price,2)))
print('You will pay around  {} this amount of tax, sorry'.format(np.round(total_tax_to_pay_approx ,2)))

fill 1TZ, form 2042 C with  19996.392200886792
fill 1UZ abbatement pour duree de detention, form 2042 C with the rebate in euro 0.0
fill 1WZ abattement de 50%, form 2042 C with the rebate in euro 955.2829898492771
fill 3VG , form 2042 C with capital gain 1335.8859339530106
fill 3VH , form 2042 C with capital loss 0 except if you have other sources of capital losses. You fill 0 because you are removing these losses from the 1TZ field
You sold for 22179.86 euros in stocks
You will pay around  12202.97 this amount of tax, sorry


# CSG deduction from previous year (2021)
WARNING WE CHANGED THE WAY TO COMPENSATE BETWEEN PLUS OR MOINS VALUE SO YOU MAY HAVE SMALL CHANGES HERE
if pre filled, keep the pre filled version

If you ahve your previous tax filling, it should be equal to your 1TZ case of last year times the deductible CSG, which is at 6.8%. If you don't, you can use this, should work. It is very likely to be pre filled though. Still there in case of need


In [25]:
if vesting_gain_before_rebate_2021 ==0:
    print('sorry, no CSG to be deducted for this year!')
    vesting_gain_after_rebate_and_moins_value_2021 =0
    deductible_csg = vesting_gain_before_rebate_2021 * tax_info_dict['deductible_csg']
else:
    vesting_gain_after_rebate_and_moins_value_2021 = vesting_gain_before_rebate_2021 - rebate_amount_2021
    deductible_csg = vesting_gain_after_rebate_and_moins_value_2021 * tax_info_dict['deductible_csg']
   

In [26]:
print('fill box 6DE', deductible_csg)

fill box 6DE 1745.7959077266146
